# Vehicular Accidents in New York City

---

## Import Libraries, Access API,  Create DataFrame and Sample Data

In [25]:
# Import dependencies
import pandas as pd
from sodapy import Socrata
import matplotlib.pyplot as plt
import numpy as np
import requests
import time
import scipy.stats as st
from scipy.stats import linregress
import json

# Import API keys & app tokens
from api_keys import api_key_regular
from api_keys import api_key_secret
from api_keys import app_token_regular
from api_keys import app_token_secret

# Create full url for API
client = Socrata("data.cityofnewyork.us", app_token_regular)

# Example authenticated client (needed for non-public datasets):
# client = Socrata(data.cityofnewyork.us,
#                  MyAppToken,
#                  username="user@example.com",
#                  password="AFakePassword")

# Return results as a JSON
results = client.get("h9gi-nx95")

# Convert to pandas DataFrame
results_values_df = pd.DataFrame.from_records(results)

# Print all column names
column_names = results_values_df.columns
for name in column_names:
    print(name)

# Sample values
results_values_df.head(20)

crash_date
crash_time
on_street_name
off_street_name
number_of_persons_injured
number_of_persons_killed
number_of_pedestrians_injured
number_of_pedestrians_killed
number_of_cyclist_injured
number_of_cyclist_killed
number_of_motorist_injured
number_of_motorist_killed
contributing_factor_vehicle_1
contributing_factor_vehicle_2
collision_id
vehicle_type_code1
vehicle_type_code2
borough
zip_code
latitude
longitude
location
cross_street_name
contributing_factor_vehicle_3
vehicle_type_code_3
contributing_factor_vehicle_4
vehicle_type_code_4
contributing_factor_vehicle_5
vehicle_type_code_5


,crash_date,crash_time,on_street_name,off_street_name,number_of_persons_injured,number_of_persons_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_cyclist_injured,number_of_cyclist_killed,...,latitude,longitude,location,cross_street_name,contributing_factor_vehicle_3,vehicle_type_code_3,contributing_factor_vehicle_4,vehicle_type_code_4,contributing_factor_vehicle_5,vehicle_type_code_5
0,2021-09-11T00:00:00.000,2:39,WHITESTONE EXPRESSWAY,20 AVENUE,2,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-03-26T00:00:00.000,11:45,QUEENSBORO BRIDGE UPPER,NaN,1,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-06-29T00:00:00.000,6:55,THROGS NECK BRIDGE,NaN,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-09-11T00:00:00.000,9:35,NaN,NaN,0,0,0,0,0,0,...,40.667202,-73.8665,"{'latitude': '40.667202', 'longitude': '-73.86...",1211 LORING AVENUE,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-12-14T00:00:00.000,8:13,SARATOGA AVENUE,DECATUR STREET,0,0,0,0,0,0,...,40.683304,-73.917274,"{'latitude': '40.683304', 'longitude': '-73.91...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2021-04-14T00:00:00.000,12:47,MAJOR DEEGAN EXPRESSWAY RAMP,NaN,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2021-12-14T00:00:00.000,17:05,BROOKLYN QUEENS EXPRESSWAY,NaN,0,0,0,0,0,0,...,40.709183,-73.956825,"{'latitude': '40.709183', 'longitude': '-73.95...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2021-12-14T00:00:00.000,8:17,NaN,NaN,2,0,0,0,0,0,...,40.86816,-73.83148,"{'latitude': '40.86816', 'longitude': '-73.831...",344 BAYCHESTER AVENUE,NaN,NaN,NaN,NaN,NaN,NaN
8,2021-12-14T00:00:00.000,21:10,NaN,NaN,0,0,0,0,0,0,...,40.67172,-73.8971,"{'latitude': '40.67172', 'longitude': '-73.897...",2047 PITKIN AVENUE,NaN,NaN,NaN,NaN,NaN,NaN
9,2021-12-14T00:00:00.000,14:58,3 AVENUE,EAST 43 STREET,0,0,0,0,0,0,...,40.75144,-73.97397,"{'latitude': '40.75144', 'longitude': '-73.973...",NaN,NaN,NaN,NaN,NaN,NaN,NaN


## DataFrame Clean-up and Hygiene

In [29]:
# Excluse columns that are unnecessary for the analysis
excluded_columns = ["on_street_name", "off_street_name", "number_of_pedestrians_injured", "number_of_pedestrians_killed", "number_of_cyclist_injured",
"number_of_cyclist_killed", "number_of_motorist_injured", "number_of_motorist_killed", "contributing_factor_vehicle_2", "vehicle_type_code2", "zip_code", "latitude", "longitude"
"location", "cross_street_name", "cross_street_name", "contributing_factor_vehicle_3", "vehicle_type_code_3", "contributing_factor_vehicle_4", "vehicle_type_code_4", "contributing_factor_vehicle_5", "vehicle_type_code_5"]
filtered_df = results_values_df.loc[:, ~results_values_df.columns.isin(excluded_columns)]

# Reorder columns with "collision_id" as the first column so that each cr
column_order = ["collision_id"] + [col for col in filtered_df.columns if col != "collision_id"]
filtered_df = filtered_df[column_order]

# Sample values
filtered_df.head(20)

# Sample values
filtered_df.head(20)

,crash_date,crash_time,number_of_persons_injured,number_of_persons_killed,contributing_factor_vehicle_1,collision_id,vehicle_type_code1,borough,longitude,location
0,2021-09-11T00:00:00.000,2:39,2,0,Aggressive Driving/Road Rage,4455765,Sedan,NaN,NaN,NaN
1,2022-03-26T00:00:00.000,11:45,1,0,Pavement Slippery,4513547,Sedan,NaN,NaN,NaN
2,2022-06-29T00:00:00.000,6:55,0,0,Following Too Closely,4541903,Sedan,NaN,NaN,NaN
3,2021-09-11T00:00:00.000,9:35,0,0,Unspecified,4456314,Sedan,BROOKLYN,-73.8665,"{'latitude': '40.667202', 'longitude': '-73.86..."
4,2021-12-14T00:00:00.000,8:13,0,0,NaN,4486609,NaN,BROOKLYN,-73.917274,"{'latitude': '40.683304', 'longitude': '-73.91..."
5,2021-04-14T00:00:00.000,12:47,0,0,Unspecified,4407458,Dump,NaN,NaN,NaN
6,2021-12-14T00:00:00.000,17:05,0,0,Passing Too Closely,4486555,Sedan,NaN,-73.956825,"{'latitude': '40.709183', 'longitude': '-73.95..."
7,2021-12-14T00:00:00.000,8:17,2,0,Unspecified,4486660,Sedan,BRONX,-73.83148,"{'latitude': '40.86816', 'longitude': '-73.831..."
8,2021-12-14T00:00:00.000,21:10,0,0,Driver Inexperience,4487074,Sedan,BROOKLYN,-73.8971,"{'latitude': '40.67172', 'longitude': '-73.897..."
9,2021-12-14T00:00:00.000,14:58,0,0,Passing Too Closely,4486519,Sedan,MANHATTAN,-73.97397,"{'latitude': '40.75144', 'longitude': '-73.973..."
